# Generative LLMs for RAG

This notebook provides a hands-on exploration of generative Large Language Models (LLMs). We'll start with cloud-based API models, and then explore how to run smaller models locally. Finally, we will explore different prompting strategies you can use to get best results.

## Model Access & Generating an API Key

For this lab, we recommend the BLABLADOR API provided by the German Supercomputing centre in Jülich. Follow [🔗 their instructions](https://sdlaml.pages.jsc.fz-juelich.de/ai/guides/blablador_api_access/) to generate an API key. Use your university login to gain access. You can also interact via their [🔗 Web UI](https://helmholtz-blablador.fz-juelich.de).

They host several models, and you can specify the following alias names in API calls:
- `alias-code` - Qwen2.5-Coder-7B-Instruct, a model that is specially trained for code.
- `alias-embeddings` - GritLM-7B, a model specially made for embeddings
- `alias-fast` - Ministral-8B-Instruct-2410, a model for high throughout (we will use this one in this lab)
- `alias-large` - DeepSeek-R1-Distill-Llama-70B, a very large model; the most accurate, but also the slowest.
- `alias-reasoning` - QwQ-32B, a model that is specially trained for reasoning.This model might not run 24h.


## Environment Setup

Make sure to install the required libraries (comment out the following line, or make sure that your environment has these dependencies installe):


In [ ]:
#!pip install openai torch transformers

In [ ]:
API_URL = "https://api.helmholtz-blablador.fz-juelich.de/v1/"
API_KEY = "<KEY>"
API_MODEL = "alias-fast" # Best for fast dev runs

## Calling LLMs via an API

For the first step, we are interacting with an LLM via a hosted API. Most providers (BLABLADOR too) follow an Open-AI compliant API, meaning that you can use the `openai` python wrapper also to query models by non-openai providers. First, we create an API client object:

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=API_URL
)

Modern LLMs are usually finetuned for instructions, and their prompting follows a turn-based pattern: each message in a conversation with the LLM has a role associated with it (`user`, the user submitting the query; `system`, general instructions at the beginning of the conversation; or `assistant`, the reply of the LLM). For our first call, we are going to use the [`completions` API endpoint](https://platform.openai.com/docs/api-reference/chat/create), which you can use in python by calling `client.chat.completions.create`.

It takes 2 mandatory arguments: the model (we are going to use `alias-fast`, saved in the `API_MODEL` variable), and the message, formatted as list of `{"role": <role>, "content": <message text>}` dictionaries.

In [ ]:
# Make a simple completion request
response = client.chat.completions.create(
    model=API_MODEL,
    messages=[
        # <your messages>
    ]
)

print(response.choices[0].message.content)

## Working with Temperature

Of course, the API allows much more parameters to influence the results of the LLM generative process. First, `temperature`. Temperature controls "randomness" in the output, where `temperature = 0` yields a deterministic result, while `temperature = 1` yields a more unstable, but usually also more creative result. A balanced choice of e.g., `temperature = 0.7` is usually used.

Try out how different temperature values affect the response generation for the same prompt:

In [ ]:
for temp in [0, 0.7, 1.0]:
    pass

## Chat Format: System, User, and Assistant Messages

Instruction-tuned LLMs generally use different message roles, which we can leverage via the API:

- **system**: sets behavior instructions, is usually passed as the first message in a chat to "set the tone"
- **user**: represents human input messages, the prompt as you would enter it in an LLM
- **assistant**: represents AI responses, the generated text received by the LLM (usually from an earlier conversation turn)

Try out different system prompts where you command the model to take on different personas to see how its reponse to the actual prompt differs.

## Multi-turn Conversations

LLMs can maintain context across multiple messages. Via the API, you can simulate conversations by appending responses and your own follow-up message to the message list. Generated responses are inserted using the `assistant` role, and the follow-up user message is then posed with the `user` role.

Ask the LLM a question, append its response, and then ask a follow-up question to see how it maintains context across the whole conversation.


## Running Local LLMs

Besides calling models via an API, you might want to run models locally, for example if you lack internet access, for experimentation without worrying about rate limits or API cost, or when working with privacy-sensitive data. In the following, we will use models loaded from the [Huggingface]() model repository and use them with the `transformers` library. As a rule of thumb, model below 400M parameters usually fit in 16GB RAM, with acceptable inference times on CPU.

We'll use a small model that can run on a CPU: [`HuggingFaceTB/SmolLM2-360M-Instruct`](). We need both the model itself, and a tokenizer to convert the message blocks into token IDs the model can consume.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"
device = "cpu" # "gpu" for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

Use the tokenizer to turn the message list into usable model inputs. The [`tokenizer.apply_chat_template`]() function can directly produce the desired result. Important: specify the arguments `tokenize=True` and `return_tensors="pt"` to get the correct Torch tensor objects for the model. Also remember to send the tokenized data to the same device as the model using the `.to(device)` method as already with the model.

Now you can call the `model.generate()` function with the tokenized inputs to produce generated text. However, the model returns token IDs, not their string representation. You can convert the model output back into human-readable format using the `tokenizer.decode()` function.

## Prompt Engineering

To get the most out of any LLM, you need to carefully design prompts. Many prompting techniques exist and are hot topic in current research. The guides linked below provide a comprehensive overview on prompt engineering. In this notebook, we will explore three prompt engineering techniques:

- Prompting-Induced Planning / Chain-of-Thought
- Self-critique
- Structured output prompting

The following guides explore prompt engineering techniques in more detail:

- [🔗 Prompt Engineering Guide](https://drive.google.com/file/d/1AbaBYbEa_EbPelsT40-vj64L-2IwUJHy/view)
- [🔗 GPT4.1 Prompting Guide](https://cookbook.openai.com/examples/gpt4-1_prompting_guide)


For prompt engineering, we will consider the RAG usecase with the example query and retrieved snippets below. The goal is to provide a short but informative answer to the user.

In [ ]:
query = "why has olive oil increased in price"
snippets = [
    ("Global olive oil prices have surged due to poor harvests in Spain and Italy, caused by extreme drought and heatwaves linked to climate change.", 0.983),
    ("A sharp decline in olive oil production, especially in major producing countries like Spain, has led to reduced supply and increased prices worldwide.", 0.942),
    ("Increased production costs, including higher labor and transportation expenses, have contributed to the rise in olive oil prices.", 0.891),
    ("Rising inflation and currency fluctuations have made imported goods, including olive oil, more expensive in several countries.", 0.847),
    ("Retailers report that consumer demand for premium oils has increased, indirectly pushing prices higher across all olive oil grades.", 0.812),
    ("Climate change has disrupted agricultural cycles in the Mediterranean region, impacting many crops including olives.", 0.768),
    ("Sunflower oil shortages due to the war in Ukraine have led to a shift in demand toward olive oil, tightening global supply.", 0.703),
    ("The Mediterranean diet, which emphasizes olive oil consumption, continues to gain popularity for its health benefits.", 0.623),
    ("Spain is one of the world's largest producers of olive oil, exporting millions of liters each year.", 0.578),
    ("Olive trees can live for hundreds of years and are cultivated mostly in Mediterranean climates.", 0.519)
]

### Vanilla RAG

The most basic RAG prompting technique directly feeds retrieved context into the model alongside the query. This approach serves as a baseline: it simply instructs the model to answer the question based on the given context, without extra guidance.

Implement the vanilla RAG prompt using a simple format with a system message and a user message containing the context and question.

### Chain-of-Thought (CoT) Prompting

Chain-of-thought prompting encourages the model to reason step by step before producing a final answer. This helps improve factual accuracy and clarity, especially when the retrieved context is complex or contains multiple causal links.

Implement CoT prompting by asking the model to first identify relevant information, then reason through the answer, and finally summarize the conclusion.

### Critique-and-Revise Prompting

To improve factuality or clarity, we can instruct the model to critique an initial answer before revising it. This multi-step prompting encourages reflection and refinement, and can lead to higher-quality outputs.

Generate an initial answer, prompt the model to critique it, and then ask for a revised version based on that critique.

### Self-Consistency

As we have seen before, a model might produce different outputs at repeated inference when using higher temperatures. We can use this to our advantage by prompting the model for self-consistency: first, we generate several candidate answers at high temperature (leveraging the more creative, diverse output), and then provide these to the model to distill into a final answer at low temperature (yielding consistent output).

Implement self-consistency by generating 3 candidates at high temperature, and combining them in a follow-up inference pass. Design special prompts for both phases.

#### Structured Output

For downstream applications, you should have the model return structured outputs, e.g., in JSON dictionaries. Then, you can elicit responses that decompose their answer into different aspects. For example, you could refine the critique-and-revise technique from before my having the model return the critique and the final response as fields in a JSON dictionary, or separate reasoning and response in the answer in order to only display the response.

For RAG, you can also use structured outputs to have the model attribute its reasoning to passages. Try to make it return its answer in a list of sentences, where for each sentence is represented as JSON dictionary with the text and the relevant passages information is taken from.